### Installing the prerequisites for using the Pinecone Vector Database

In [ ]:


!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
# this tiktoken helps us to call the openAI embedding
!pip install tiktoken
pip install -U langchain-community

In [37]:
pip install sentence-transformers

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB 1.3 MB/s eta 0:02:05
   ---------------------------------------- 0.2/159.8 MB 3.0 MB/s eta 0:00:53
   ---------------------------------------- 1.2/159.8 MB 8.4 MB/s eta 0:00:19
   ---------------------------------------- 1.7/159.8 MB 9.9 MB/s eta 0:00:16
    --------------------------------------- 3


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
pip install transformers



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Generating API Key in pinecone for using the vectordatabase of pinecone

Open the Pinecone webiste 

https://www.google.com/search?q=pincone.

- Get sign up 
- since we are using the free tier account of pinecone so we can able to create only 1 Index which helps us to generate the pinecone API key
- click on API key where we can see the default API key and generated API by creation of index 
- By using any one API key bydefault or generated API key by creation if index we can able to use the vector database of pinecone

- copy any one of the API key of pinecone in API key section 



### Importing the necessary modules from langchain because langchain framework acts as wrapper in OPenAI ,pinecone,huggingface etc langchain consist of everything which are required  to develop the generative AI Application 

In [43]:

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

### Lets create the Folder in the workspace where we can storing the data which we furthur convert the data into vector embedding and store in pinecone vector database and furthur we pass it to llm model 

In [2]:
mkdir pdfs

In [15]:
loader=PyPDFDirectoryLoader('pdfs')  # by using PyPDFDirectoryLoader i could able to load the transformer research paper which is present in pdf folder and saved the attention is all you need research pdf data in loader variable

In [19]:
data=loader.load()
print(data)  # as we can see the output we successfully loaded the pdf data and saved in data variable

[Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring

### Performing the Tokenization on data by using RecursiveCharacterTextSplitter()

RecursiveCharacterTextSplitter helps us to split the pdf data content in the form of Chunks  , here chunks stands for small paragraph which has meaningful context so like this way we can divide the whole paragraph chunk into mentioned desired no of chunks (paragraphs) by using this RecursiveCharacterTextSplitter 

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)  # By using RecursiveCharacterTextSplitter class and passing the 2 arguments iam able to perform the tokenization and i have created the object for this class

In [21]:
text_chunks = text_splitter.split_documents(data)  # with the help of the created object for the class iam calling one of the class method which is split_document which perform 

In [22]:
text_chunks

[Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or', metadata={'source': 'pdfs\\attention-is-all-you-need-Research Paper.pdf', 'page': 0}),
 Document(page_content='convolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two mach

In [24]:
len(text_chunks)  # the total chunks or smaller paragraphs which our methid split_documents converted into 76 chunks or smaller documents or paragraph

76

In [25]:
print(text_chunks[0].page_content)   # acessing the each chunk or document or paragraph by index value of list pf text_chunks

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or


In [28]:
print(text_chunks[2].page_content) 

less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.0 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature.
1 Introduction


In [29]:
print(text_chunks[75].page_content)  # so like this way we can access the max index of 75 which is our last chunk or last small paragraph of the pdf data 

[31] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang
Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine
translation system: Bridging the gap between human and machine translation. arXiv preprint
arXiv:1609.08144 , 2016.
[32] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation. CoRR , abs/1606.04199, 2016.
11


## Fetching the Pre trained vector embedded transformer from Huggingface application  (Encoding Techinque with Pretrained model)
- Setting up the Huggingface API KEY for using the Pretrained vector Embedding Model for converting the text data into embedding dense vectors or continues vector space  to capture the semantic meaning between the each feature the pretrained trained on vs each input text data which we have in our pdf 


In [30]:
from dotenv import load_dotenv  # Python-dotenv reads key-value pairs from a .env file and can set them as environment variables., we use this library to read the open AI api key secretly rather than exposing it to public it helps us to read the key in secrete and than after mentioning the key value pair of openai key in .env file makesure add this .env file to .gitiognore so it wont get push to github

load_dotenv()  # take environment variables from .env.

True

In [60]:
# reading the hugginface api key token from .env file 
KEY1=os.getenv("HUGGINGFACEHUB_API_TOKEN")
#KEY2=os.getenv("OpenAI_API_Key")
KEY2=""

In [61]:
embedding1=OpenAIEmbeddings(openai_api_key=KEY2)

In [63]:
embedding1.embed_query("How are you")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [41]:
from sentence_transformers import SentenceTransformer

# Load the pretrained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Example usage: encode a list of sentences to get their embeddings
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
x=len(embeddings)
print(x)


c:\Users\mahen\anaconda_new\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2


### Saving the converted embedded dense vector in the pinecode vectordatabase

Open the Pinecone webiste 

https://www.google.com/search?q=pincone.

- Get sign up 
- click on API key where we can see the default API key and generated API key by clicking on new option
- By using any one API key bydefault or generated API key we can able to use the vector database of pinecone to store the embedded vector

- copy any one of the API key and API KEY name in pinecone website of API key section 

In [67]:
# fetching the unique api key of pine cone which helps us to save the embedded dense vector in this pinecone vector database
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'aa069090-c8bc-4997-bca6-c68594ab4a57')  # API Key value
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'default')  # api key name 

### Initilizing the Pinecone vector Database connection

In [66]:
import pinecone

In [70]:
# calling the init method for initilizing the conenction of pine cone vector database
pinecone.init(api_key=PINECONE_API_KEY,  # find at app.pinecone.io
     environment=PINECONE_API_ENV  # next to api key in console
)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



#### Pinecone is being running on top of cloud server so we can use pinecone server which is fully managed by AWS, Azure, GCP
- Just by creating the Index in the pinecone web application 

In [71]:
index_name = "testing" # put in the name of your pinecone index here